# Credit Risk with Quantum Amplitude Estimation (QAE)

> Contact : Ziyuan Li (stan.ziyuan.li@hsbc.com.cn) 

```
************************************************
Goal of this technotes:
    1. Know the idea of credit risk and general approach to perform analysis with quantum algorithm
    2. Know how to prepare the quantum circuit for this task
    3. Know how to run Credit Risk with Qiskit
************************************************   
```

## Motivation
The economic capital requirement is the difference between the value at risk and the expected loss. It is designed to cover losses between the expected loss and the value at risk at a given confidence level. Extreme losses, i.e., those beyond the value at risk, are not covered by the economic capital requirement

<img src="https://raw.githubusercontent.com/ZiyuanLi007/quantumalg/main/technotes/images/cr/qcalg_cr_ecr.png" width="500">

For a portfolio of $K$ assets the multivariate random variable $(L_1, L_2, ..., L_K)$ denotes each possible loss associated to each asset. The expected value of the total loss is $E[L] = \sum_{k=1}^K E[L_K]$. The value at risk (VaR) for a given confidence level $\alpha$ is defined as the smallest total loss that still has a probability greater than or equal to $\alpha$, i.e.,

$$ \text{VaR}_{\alpha}(L) = \inf_{x \ge 0} \{ x \mid \mathbb{P}[L <= x] \geq \alpha \}$$

The economic capital requirement at confidence level $\alpha$ is thus defined as the difference between the value at risk and the expected loss

$$ \text{ECR}_{\alpha}(L) = \text{VaR}_{\alpha}(L) - E[L] $$

Quantum amplitude estimation algorithm can be used to estimate financial risk measures such as the value at risk and the conditional value at risk with a quadratic speedup compared to classical Monte Carlo simulations.



## 1. Conditional Independence Model

For simplicity, if  we assume that all losses are independent and can be expressed as $L_k = \lambda_k X_k$, where $\lambda_k > 0$ is the loss given default (LGD) and $X_k \in \{0,1\}$ is corresponding Bernoulli random variable. The probability that $X_k = 1$, i.e., a loss for asset k, is $p_k$. The expected total loss is given by $E[L] = \sum_{k=1}^K \lambda_k p_k$. 

In practice, asset defaults are often correlated, especially in periods of financial stress. The defaults $X_k$ are no longer independent but follow a condi- tional independence scheme, i.e., given a value $z$ sampled from a latent random variable $Z$ following a standard normal distribution, the default probability of asset $k$ is given by

$$p_k(z) = F\left( \frac{F^{-1}(p_k^0) - \sqrt{\rho_k}z}{\sqrt{1 - \rho_k}} \right) $$

where $F$ denotes the cumulative distribution function of $Z$, $p_k^0$ is the default probability of asset $k$ for $z=0$ and $\rho_k$ is the sensitivity of the default probability of asset $k$ with respect to $Z$. Thus, given a concrete realization of $Z$ the individual default events are assumed to be independent from each other. The expected loss is

$$ E[L] = \int_{z=-\infty}^{\infty} \sum_{k=1}^K \lambda_k p_k(z) f(z) dz $$

where $f(z)$ is the PDF function of $Z$

## 2. Quantum Circuit Construction
To apply quantum computation, the primary task is to load the correlated default risk for each asset of the portfolio into the quantum circuit.

<h3><center> High level circuit of the operator $A$</center></h3>
<img src="https://raw.githubusercontent.com/ZiyuanLi007/quantumalg/main/technotes/images/cr/qcalg_cr_Acircuit.png" width="600">

The first qubit register with $n_Z$ qubits represents the latent random variable $Z$, the second qubit register with $K$ qubits represents the Bernoulli random variables $X_k$ of each asset, the third qubit register with $n_S$ qubits represents the sum of the losses. Then it comes to the comparator operator $C$ and the last qubit is flipped to $|1\rangle$ if the total loss is less than or equal to a given $x$



### Step 1. Load  Z distribution using operator $U_Z$
We use $n_z$ qubits to discretize the distribution to $2^{n_z}$ slots. 

<img src="https://raw.githubusercontent.com/ZiyuanLi007/quantumalg/main/technotes/images/cr/qcalg_cr_UZ.png" width="300">

Operator $U_Z$ inputs the state $|0\rangle^{\otimes n_z}$ for these $n_z$ qubits, and output $|\psi\rangle_{U_Z}$, a superposition of $2^{n_z}$ entangled states:

$$|\psi\rangle_{U_Z} = \sum_{z=0}^{2^{n_z}-1} \sqrt{f(z)} |z\rangle$$

### Step 2. Load  uncorrelated default using operator using operator $L_X$
When the default events $\{X_1, X_2, ..., X_K\}$ are uncorrelated, we can encode the $X_k$ of each asset in the state of a corresponding qubit by applying to qubit $k$ a Y-rotation $R_Y(\theta_p^k)$ with angle $\theta_p^k = 2 \arcsin(\sqrt{p_k^0})$. The operator $L_X$ involves $n_x$ qubits to load the $n_x$ independent assets. For each of the $n_x$ qubits, operator $L_X$ inputs the initial state $|0\rangle^{\otimes n_x}$ and outputs the state:

$$|\psi\rangle_{L_X} = (\sqrt{1-p_k^0}|0\rangle + \sqrt{p_k^0}|1\rangle)^{\otimes n_x}$$

so that the probability for state $|1\rangle$ encodes the default probability $p_k^0$ 

### Step 3. Load correlated default using operator $L_Z$
The qubit register representing $Z$ is then used to control the rotation angles $\theta_p^k(z) = 2 \arcsin(\sqrt{p_k(z)})$ that prepare the qubits representing the $X_k$. For simplicity, a first order approximation of $\theta_p^k(z)$ and include the affine mapping from $z$ to $i$ ($i \in \{0, ..., 2^{n_z}-1\}$), i.e. $\theta_p^k(z) \approx a_k i + b_k$.

The qubit representing Xk is prepared using Y -rotations controlled by the qubits representing Z

<h3><center>The qubit representing $X_k$ is prepared using Y-rotations controlled by the qubits representing Z.</center></h3>
<img src="https://raw.githubusercontent.com/ZiyuanLi007/quantumalg/main/technotes/images/cr/qcalg_cr_LZ.png" width="600">

In short, operator $L_Z$ inputs $\sum_{z=0}^{2^{n_z}-1} \sqrt{f(z)} |z\rangle$ and $(\sqrt{1-p_k^0}|0\rangle + \sqrt{p_k^0}|1\rangle)^{\otimes n_x}$ from $U_Z$ and $U_X$ and outputs the further entangled state:

$$|\psi\rangle_{L_Z} = \sum_{z=0}^{2^{n_z}-1} \sqrt{f(z)} |z\rangle (\sqrt{1-p_k(z)}|0\rangle + \sqrt{p_k(z)}|1\rangle)^{\otimes n_x}$$


### Step 4. Load total loss using operator $S$



### Step 5. Load  tranche loss using operator $C\&R$

